What I tried to do before in Still Public Method (1) is taking so long. The methodology I employed is to build a new dataframe from scratch.

What I should be doing though, is to drop rows from compustat. So in Still Public Method (2), we employ an alternative method:

1. Display Unique
2. Find if there are rows with the same 'Global Company Keys'.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
unique = pd.read_csv('/content/drive/MyDrive/Senior Fall/Senior Thesis/Data/Separately/Unique.csv', converters={'Global Company Key':str})

In [ ]:
compustat = pd.read_csv('/content/drive/MyDrive/Senior Fall/Senior Thesis/Data/Separately/Compustat.csv', converters={'Global Company Key':str}, dtype='string')

<ipython-input-3-64929dbe97d8>:1: ParserWarning: Both a converter and dtype were specified for column Global Company Key - only the converter will be used.
  compustat = pd.read_csv('/content/drive/MyDrive/Senior Fall/Senior Thesis/Data/Separately/Compustat.csv', converters={'Global Company Key':str}, dtype='string')


In [ ]:
unique.drop_duplicates()
unique = unique.sort_values(by=['Global Company Key', 'Data Date'], ascending=[True, False])
unique

,Global Company Key,Data Date,Ticker Symbol,CUSIP,Company Name,Accounting Changes - Cumulative Effect,Accumulated Other Comprehensive Income (Loss),Current Assets - Other - Total,Current Assets - Total,Other Long-term Assets,...,Implied Option Expense Preliminary.1,Implied Option Expense.1,Research and Development Expense.1,"Selling, General and Administrative Expenses.1",CIK Number,Active/Inactive Status Marker,Common Shares Traded - Quarter,Dividends per Share - Pay Date - Quarter,Div per Share - Exdate - Quarter,Market Value - Total
316,001086,1997-03-31,3AMETE,031095102,AMETECH INC,0.0,NaN,1.272,5.093,NaN,...,NaN,NaN,NaN,0.477,1969.0,I,123700.0,0.0000,0.0000,NaN
257,001266,2013-09-30,ALCO,016230104,ALICO INC,0.0,0.000,1.283,59.795,2.629,...,0.0,0.0,NaN,7.923,3545.0,A,670923.0,0.0800,0.0800,300.7057
58,001633,2018-01-31,ALOG,032657207,ANALOGIC CORP,0.0,-0.546,9.316,391.093,8.364,...,NaN,NaN,30.642,79.628,6284.0,I,4273726.0,0.1000,0.1000,1035.7526
338,001743,1988-03-31,ARCB,03937C105,ARCBEST CORP,NaN,NaN,9.686,128.213,NaN,...,NaN,NaN,NaN,NaN,894405.0,A,1868800.0,0.0900,0.0900,NaN
86,001920,2015-09-30,AVP,054303102,AVON PRODUCTS,0.0,-1417.400,443.900,2385.600,77.000,...,0.0,NaN,NaN,2540.200,8868.0,A,615526931.0,0.0600,0.0600,1415.0435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,190959,2015-06-30,LMOS,550283105,LUMOS NETWORKS CORP,0.0,-12.067,10.905,66.811,7.059,...,NaN,0.0,NaN,38.591,1520744.0,I,9418663.0,0.0000,0.0000,338.4248
24,196355,2020-12-31,SYNC,871561106,SYNACOR INC,0.0,-0.558,3.091,26.357,0.593,...,NaN,NaN,12.007,37.490,1408278.0,I,4083206.0,0.0000,0.0000,53.9158
28,197559,2020-09-30,EIGI,29272B105,ENDURANCE INTL GRP HLDGS INC,0.0,-1.965,128.342,306.902,77.946,...,NaN,NaN,79.958,362.972,1237746.0,I,56959834.0,0.0000,0.0000,812.2502
73,241159,2016-09-30,CHT,17133Q502,CHUNGHWA TELECOM LTD,0.0,-19.598,204.960,2244.715,355.786,...,NaN,NaN,89.164,778.605,1132924.0,A,13728247.0,1.7304,1.7304,NaN


In [ ]:
def rows_after_date(date, df):
  return df[df['Data Date'] >= date]

def rows_matching_gvkey(gvkey, df):
  return df.loc[df['Global Company Key'] == gvkey]

For each row in unique:
- store gvkey
- store data date

we're gonna locate all rows_after_date of the rows_matching_gvkey(gvkey, compustat). And then drop the three quarters before along with this one.

In [ ]:
count = 1
for index, row in unique.iterrows():
  print(count)
  count += 1
  indices = rows_matching_gvkey(row['Global Company Key'], compustat).index
  compustat = compustat.drop(index = indices)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
compustat.to_csv('firms_dropped.csv', index=False)

In [ ]:
compustat

,Global Company Key,Data Date,Ticker Symbol,CUSIP,Company Name,Accounting Changes - Cumulative Effect,Accumulated Other Comprehensive Income (Loss),Current Assets - Other - Total,Current Assets - Total,Other Long-term Assets,...,Implied Option Expense Preliminary.1,Implied Option Expense.1,Research and Development Expense.1,"Selling, General and Administrative Expenses.1",CIK Number,Active/Inactive Status Marker,Common Shares Traded - Quarter,Dividends per Share - Pay Date - Quarter,Div per Share - Exdate - Quarter,Market Value - Total
0,001004,1981-02-28,AIR,000361105,AAR CORP,<NA>,<NA>,1.7570,57.1350,<NA>,...,<NA>,<NA>,<NA>,<NA>,0000001750,A,307900.0000,0.1000,0.1000,<NA>
1,001004,1981-05-31,AIR,000361105,AAR CORP,<NA>,<NA>,1.2850,59.3340,<NA>,...,<NA>,<NA>,<NA>,<NA>,0000001750,A,131200.0000,0.1100,0.1100,<NA>
2,001004,1981-08-31,AIR,000361105,AAR CORP,<NA>,<NA>,1.5790,82.8010,<NA>,...,<NA>,<NA>,<NA>,5.8050,0000001750,A,200900.0000,0.1100,0.1100,<NA>
3,001004,1981-11-30,AIR,000361105,AAR CORP,<NA>,<NA>,<NA>,77.5110,<NA>,...,<NA>,<NA>,<NA>,<NA>,0000001750,A,471800.0000,0.1100,0.1100,<NA>
4,001004,1982-02-28,AIR,000361105,AAR CORP,<NA>,<NA>,2.8130,71.8700,<NA>,...,<NA>,<NA>,<NA>,<NA>,0000001750,A,386700.0000,0.1100,0.1100,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670861,353945,2022-09-30,ACLLY,00449R109,ACCELLERON INDUSTRIES AG,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,0001948862,A,<NA>,<NA>,<NA>,<NA>
670862,353945,2022-12-31,ACLLY,00449R109,ACCELLERON INDUSTRIES AG,0.0000,63.5210,8.3710,614.9040,178.5670,...,<NA>,<NA>,51.0870,200.6890,0001948862,A,916951.0000,0.0000,0.0000,<NA>
670863,354003,2023-03-31,BEMB,46436E262,ISHARES JP MORGAN B U E M B,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,A,<NA>,<NA>,<NA>,<NA>
670864,354003,2023-06-30,BEMB,46436E262,ISHARES JP MORGAN B U E M B,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,A,<NA>,<NA>,<NA>,<NA>
